In [ ]:
# Install required packages (if needed)
# ! pip install llama-stack-client python-dotenv rich termcolor


In [28]:
# Import required libraries
import os
import json
from dotenv import load_dotenv
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger
from termcolor import cprint


In [ ]:
# Load configuration
load_dotenv('config.env')

# Get configuration from environment
base_url = os.environ.get('LLAMA_STACK_URL')
model_id = os.environ.get('LLM_MODEL_ID')
temperature = float(os.environ.get("TEMPERATURE", 0.0))
max_tokens = int(os.environ.get("MAX_TOKENS", 4096))

# Sampling parameters
sampling_params = {
    "strategy": {"type": "greedy"},
    "max_tokens": max_tokens,
    "temperature": temperature,
}

print(f"✅ Configuration loaded:")
print(f"  Llama Stack URL: {base_url}")
print(f"  Model ID: {model_id}")
print(f"  Temperature: {temperature}")
print(f"  Max Tokens: {max_tokens}")


In [31]:
# Initialize Llama Stack client
client = LlamaStackClient(base_url=base_url)

print(f'✅ Connected to Llama Stack server at {base_url}')
print(f'✅ Using model: {model_id}')

# Check available Atlassian MCP tools
print("\n🔍 Checking Atlassian MCP tools:")
try:
    atlassian_tools = client.tools.list(toolgroup_id="mcp::atlassian")
    print(f"✅ Found {len(atlassian_tools)} Atlassian tools")
    
    # Find Confluence tools
    confluence_tools = [t for t in atlassian_tools if 'confluence' in str(t).lower()]
    print(f"🔍 Confluence tools: {len(confluence_tools)}")
    for tool in confluence_tools:
        tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
        print(f"   - {tool_name}")
        
except Exception as e:
    print(f"❌ Error loading tools: {e}")


✅ Connected to Llama Stack server at http://localhost:8321

✅ Using model: r1-qwen-14b-w4a16

🔍 Checking Atlassian MCP tools:

DEBUG:llama_stack_client._base_client:Request options: {'method': 'get', 'url': '/v1/tools', 'params': {'toolgroup_id': 'mcp::atlassian'}, 'post_parser': <function DataWrapper._unwrapper at 0x111874400>}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Aatlassian
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8321 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x11302fd10>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'

✅ Found 37 Atlassian tools

...

- confluence_get_labels

- confluence_get_page

- confluence_get_page_children

- confluence_search

- confluence_update_page

In [32]:
# Create simple agent for Confluence search
agent = ReActAgent(
    client=client,
    model=model_id,
    instructions="You are a helpful assistant that can search Confluence pages. Use the available Confluence search tools to find pages in the OCP space.",
    tools=["mcp::atlassian"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params=sampling_params,
)

print("✅ Simple Confluence search agent created")


DEBUG:llama_stack_client._base_client:Request options: {'method': 'post', 'url': '/v1/agents', 'files': None, 'idempotency_key': 'stainless-python-retry-9d792d1d-ca0c-4efb-8c68-ba37a7bdea0f', 'json_data': {'agent_config': {'model': 'r1-qwen-14b-w4a16', 'instructions': 'You are a helpful assistant that can search Confluence pages. Use the available Confluence search tools to find pages in the OCP space.', 'toolgroups': ['mcp::atlassian'], 'client_tools': [], 'response_format': {'type': 'json_schema', 'json_schema': {'$defs': {'Action': {'properties': {'tool_name': {'title': 'Tool Name', 'type': 'string'}, 'tool_params': {'items': {'$ref': '#/$defs/Param'}, 'title': 'Tool Params', 'type': 'array'}}, 'required': ['tool_name', 'tool_params'], 'title': 'Action', 'type': 'object'}, 'Param': {'properties': {'name': {'title': 'Name', 'type': 'string'}, 'value': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'type': 'number'}, {'type': 'boolean'}], 'title': 'Value'}}, 'required': ['name',

✅ Simple Confluence search agent created

In [33]:
# Simple test function
def test_confluence_search(query, space="OCP"):
    """Test Confluence search with a simple query"""
    print(f"\n🔍 Testing Confluence search: '{query}' in space '{space}'")
    print("="*60)
    
    try:
        # Create a session
        session_id = agent.create_session("confluence-test")
        
        # Create a turn with the search request
        response = agent.create_turn(
            messages=[{"role": "user", "content": f"Search for '{query}' in Confluence space '{space}'"}],
            session_id=session_id,
            stream=True
        )
        
        # Process the streaming response
        for log in EventLogger().log(response):
            log.print()
            
        return session_id
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None


In [38]:
## Test Confluence Search

test_confluence_search("Out of Memory Error", space="OCP")

🔍 Testing Confluence search: 'Out of Memory Error' in space 'OCP'

============================================================

DEBUG:llama_stack_client._base_client:Request options: {'method': 'post', 'url': '/v1/agents/d3f18cb4-f4a7-4cb3-a762-a9ea5147a25d/session', 'files': None, 'idempotency_key': 'stainless-python-retry-60db30f7-de1f-4fdf-a63b-12156f55c599', 'json_data': {'session_name': 'confluence-test'}}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: POST http://localhost:8321/v1/agents/d3f18cb4-f4a7-4cb3-a762-a9ea5147a25d/session
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8321 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x11330cd70>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.

🚀 Starting turn with 1 message(s)

Session ID: d11a3bb0-a534-414e-9e5e-4e89ad17bbdb

...

📝 Input Message 1:

Role: user

Content: Search for 'Out of Memory Error' in Confluence space 'OCP'

DEBUG:llama_stack_client._base_client:Request options: {'method': 'post', 'url': '/v1/agents/d3f18cb4-f4a7-4cb3-a762-a9ea5147a25d/session/d11a3bb0-a534-414e-9e5e-4e89ad17bbdb/turn', 'files': None, 'idempotency_key': 'stainless-python-retry-cc8ac8a4-3b95-4aa2-8f74-db8bea68414a', 'json_data': {'messages': [{'role': 'user', 'content': "Search for 'Out of Memory Error' in Confluence space 'OCP'"}], 'documents': None, 'stream': True, 'toolgroups': True}}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: POST http://localhost:8321/v1/agents/d3f18cb4-f4a7-4cb3-a762-a9ea5147a25d/session/d11a3bb0-a534-414e-9e5e-4e89ad17bbdb/turn
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:r

❌ Error: Error code: 400 - {'error': {'detail': {'errors': [{'loc': ['body', 'toolgroups'], 'msg': 'Input should 
be a valid list', 'type': 'list_type'}]}}}

In [34]:
## Direct Tool Call Test


In [35]:
# Test direct tool call (bypass agent)
def test_direct_tool_call():
    """Test calling Confluence tools directly"""
    print("\n🔧 Testing direct tool call...")
    
    try:
        # Get available tools
        atlassian_tools = client.tools.list(toolgroup_id="mcp::atlassian")
        print(f"Available tools: {len(atlassian_tools)}")
        
        # Find a Confluence search tool
        confluence_tool = None
        for tool in atlassian_tools:
            tool_name = getattr(tool, 'name', getattr(tool, 'identifier', str(tool)))
            if 'confluence' in str(tool).lower() and 'search' in str(tool).lower():
                confluence_tool = tool_name
                break
        
        if confluence_tool:
            print(f"Found Confluence tool: {confluence_tool}")
            
            # Try to call it directly
            result = client.tools.call(
                tool_name=confluence_tool,
                tool_params=[
                    {"name": "space", "value": "OCP"},
                    {"name": "query", "value": "Out of Memory Error"}
                ]
            )
            print(f"✅ Direct tool call successful!")
            print(f"Result: {result}")
        else:
            print("❌ No Confluence search tool found")
            
    except Exception as e:
        print(f"❌ Direct tool call failed: {e}")

# Run the direct test
test_direct_tool_call()


🔧 Testing direct tool call...

DEBUG:llama_stack_client._base_client:Request options: {'method': 'get', 'url': '/v1/tools', 'params': {'toolgroup_id': 'mcp::atlassian'}, 'post_parser': <function DataWrapper._unwrapper at 0x111874400>}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Aatlassian
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8321 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1132e2d00>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'GET']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'GET']>
DEBUG:htt

Available tools: 37

Found Confluence tool: confluence_search

❌ Direct tool call failed: 'ToolsResource' object has no attribute 'call'

In [36]:
# Test 2.1: Simple Jira Incident Creation (Using Improved Agent)
print("🎫 TEST 2.1: Simple Jira Incident Creation (Improved Agent)")
print("="*70)

simple_jira_test = """
Create a simple test Jira incident in the KAN project with these parameters:
- project_key: "KAN"
- summary: "Test Incident - MCP Atlassian Integration Test"
- issue_type: "Task"
- description: "This is a test incident to verify Jira MCP integration is working correctly"
- additional_fields: {}

Use the correct dictionary format for additional_fields, not a JSON string.
"""

run_test_session(improved_test_agent, "jira-simple-incident-improved", simple_jira_test)


🎫 TEST 2.1: Simple Jira Incident Creation (Improved Agent)

======================================================================

DEBUG:llama_stack_client._base_client:Request options: {'method': 'post', 'url': '/v1/agents/548ba986-cb5e-4e78-aba3-f90bde282001/session', 'files': None, 'idempotency_key': 'stainless-python-retry-2741bb54-156a-4165-b179-62c0bc1b2316', 'json_data': {'session_name': 'test-jira-simple-incident-improved'}}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: POST http://localhost:8321/v1/agents/548ba986-cb5e-4e78-aba3-f90bde282001/session
DEBUG:httpcore.connection:close.started
DEBUG:httpcore.connection:close.complete
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=8321 local_address=None timeout=5.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x113354f50>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11

\n🧪 Running test: jira-simple-incident-improved

============================================================

...

📝 Input Message 1:

Role: user

Content: 
Create a simple test Jira incident in the KAN project with these parameters:
- project_key: "KAN"
- summary: "Test Incident - MCP Atlassian Integration Test"
- issue_type: "Task"
- description: "This is a test incident to verify Jira MCP integration is working correctly"
- additional_fields: {}

Use the correct dictionary format for additional_fields, not a JSON string.

DEBUG:llama_stack_client._base_client:Request options: {'method': 'post', 'url': '/v1/agents/548ba986-cb5e-4e78-aba3-f90bde282001/session/a738314f-1a69-4507-9a81-a0903c802751/turn', 'files': None, 'idempotency_key': 'stainless-python-retry-14baa8e9-43dc-4adb-854f-6f0c21dea6f8', 'json_data': {'messages': [{'role': 'user', 'content': '\nCreate a simple test Jira incident in the KAN project with these parameters:\n- project_key: "KAN"\n- summary: "Test Incident - MCP Atlassian Integration Test"\n- issue_type: "Task"\n- description: "This is a test incident to verify Jira MCP integration is working correctly"\n- additional_fields: {}\n\nUse the correct dictionary format for additional_fields, not a JSON string.\n'}], 'documents': None, 'stream': True, 'toolgroups': True}}
DEBUG:llama_stack_client._base_client:Sending HTTP Request: POST http://localhost:8321/v1/agents/548ba986-cb5e-4e78-aba3-f90bde282001/session/a738314f-1a69-4507-9a81-a0903c802751/turn
DEBUG:httpcore.http11:send_request_he

BadRequestError: Error code: 400 - {'error': {'detail': {'errors': [{'loc': ['body', 'toolgroups'], 'msg': 'Input should be a valid list', 'type': 'list_type'}]}}}

In [ ]:
# Debug: Test with explicit toolgroups parameter to fix the 400 error
def test_with_explicit_toolgroups():
    """Test with explicit toolgroups parameter to fix the 'Input should be a valid list' error"""
    print("\n🔧 Testing with explicit toolgroups parameter...")
    
    try:
        # Create a session
        session_id = agent.create_session("debug-test")
        print(f"✅ Session created: {session_id}")
        
        # Try to create a turn with explicit toolgroups
        # This should fix the 'Input should be a valid list' error
        response = agent.client.agents.turn.create(
            agent_id=agent.agent_id,
            session_id=session_id,
            messages=[{"role": "user", "content": "Search for 'test' in Confluence space 'OCP'"}],
            stream=True,
            toolgroups=["mcp::atlassian"]  # Explicit list instead of True
        )
        
        print("✅ Turn created successfully with explicit toolgroups!")
        
        # Process the streaming response
        for log in EventLogger().log(response):
            log.print()
            
        return session_id
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Run the debug test
test_with_explicit_toolgroups()
